In [10]:
!pip install PyPDF2
!pip install keybert
!pip install requests

In [9]:
import requests
from PyPDF2 import PdfReader
import re
from keybert import KeyBERT

# Function to extract skills from a PDF resume using KeyBERT
def extract_skills_from_resume(pdf_path):
    try:
        # Create a PdfReader object
        reader = PdfReader(pdf_path)

        # Extract text from all pages
        all_text = ""
        for page in reader.pages:
            all_text += page.extract_text()

        # Define a comprehensive list of technical skills
        comprehensive_skills = [
            'Python', 'Java', 'JavaScript', 'C++', 'C#', 'C', 'Ruby', 'Swift',
            'Kotlin', 'TypeScript', 'Go', 'Rust', 'PHP', 'Scala', 'Perl',
            'R', 'MATLAB', 'Dart', 'Haskell', 'Lua', 'Groovy',
            'HTML', 'CSS', 'React', 'Angular', 'Vue', 'Django', 'Flask',
            'Node.js', 'Express', 'Spring Boot', 'Laravel', 'Ruby on Rails',
            'SQL', 'MySQL', 'PostgreSQL', 'MongoDB', 'SQLite', 'Oracle',
            'AWS', 'Azure', 'GCP', 'Google Cloud', 'Kubernetes', 'Docker',
            'Git', 'Jenkins', 'Ansible', 'Terraform', 'CI/CD', 'Maven',
            'NumPy', 'Pandas', 'TensorFlow', 'PyTorch', 'Keras', 'NLP',
            'Linux', 'Windows', 'MacOS', 'Ubuntu'
        ]

        # Use KeyBERT to extract keywords from the resume
        kw_model = KeyBERT()
        keywords = kw_model.extract_keywords(all_text, keyphrase_ngram_range=(1, 3), stop_words='english', top_n=100)

        # Extract skills based on direct matches with the comprehensive list
        skills = []
        for skill in comprehensive_skills:
            if re.search(r'\b' + re.escape(skill) + r'\b', all_text, re.IGNORECASE):
                skills.append(skill)

        # Optionally, add extracted keywords from KeyBERT that match skills
        for keyword, _ in keywords:
            if any(skill.lower() in keyword.lower() for skill in comprehensive_skills):
                skills.append(keyword)

        # Remove duplicates while preserving order
        unique_skills = list(dict.fromkeys(skills))

        # Return top 5 skills to avoid overloading the API request
        return unique_skills[:5]

    except FileNotFoundError:
        print("Error: The file was not found. Please check the file path and try again.")
        return []
    except Exception as e:
        print(f"An error occurred: {e}")
        return []

# Function to search for job recommendations using Adzuna API
# Function to search for job recommendations using Adzuna API
def get_job_recommendations(skills, country):
    app_id = "55ce14f0"  # Your API ID
    app_key = "b0f6219c55413f59a42a924e8286af7c"  # Your API Key

    # Base URL for Adzuna API (adjusted for the selected country)
    base_url = f"http://api.adzuna.com/v1/api/jobs/{country}/search/1"

    # Broader search query to increase matches
    skill_query = "Software Developer"  # Broadened query

    # Parameters for the Adzuna API request
    params = {
        "app_id": app_id,
        "app_key": app_key,
        "results_per_page": 20,  # Increased number of results
        "what": skill_query,
        "content-type": "application/json"
    }

    # Make the API request
    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        job_data = response.json()
        if 'results' in job_data:
            jobs = job_data['results']
            if jobs:
                print(f"Found {len(jobs)} job(s) matching your skills in {country.upper()}:\n")
                for job in jobs:
                    print(f"Title: {job['title']}")
                    print(f"Company: {job['company']['display_name']}")
                    print(f"Location: {', '.join(job['location']['area'])}")
                    print(f"Link: {job['redirect_url']}\n")
            else:
                print("No job listings found for the given skills.")
        else:
            print("Error fetching job data. Check the response for details.")
    else:
        print(f"Failed to retrieve jobs: {response.status_code}")
        print(f"Response content: {response.text}")  # Debugging: Print raw response

# Main function to process the resume and fetch job recommendations
def main():
    pdf_path = input("Enter the path to your PDF file: ")
    country = input("Enter the country code (e.g., 'in' for India, 'us' for the United States, 'uk' for the United Kingdom): ").strip().lower()

    skills = extract_skills_from_resume(pdf_path)

    if skills:
        print(f"Skills extracted from your resume: {skills}\n")
        get_job_recommendations(skills, country)
    else:
        print("No skills found in the resume.")


# Main function to process the resume and fetch job recommendations
def main():
    pdf_path = input("Enter the path to your PDF file: ")
    country = input("Enter the country code (e.g., 'in' for India, 'us' for the United States, 'uk' for the United Kingdom): ").strip().lower()

    skills = extract_skills_from_resume(pdf_path)

    if skills:
        print(f"Skills extracted from your resume: {skills}\n")
        get_job_recommendations(skills, country)
    else:
        print("No skills found in the resume.")

if __name__ == "__main__":
    main()


Enter the path to your PDF file: /content/Deedy_Resume_Reversed.pdf
Enter the country code (e.g., 'in' for India, 'us' for the United States, 'uk' for the United Kingdom): au
Skills extracted from your resume: ['Python', 'JavaScript', 'C', 'PHP', 'R']

Found 20 job(s) matching your skills in AU:

Title: Developer Programmer / Software Engineer
Company: Nexus Innovate Pty Ltd
Location: Australia, Victoria, Melbourne Region
Link: https://www.adzuna.com.au/details/4978917898?utm_medium=api&utm_source=55ce14f0

Title: Software Developer
Company: Life is Sweet Bakery Pty Ltd trading as Cupcake Central Bakery 
Location: Australia, Victoria, Melbourne Region, Maribyrnong Area, West Footscray
Link: https://www.adzuna.com.au/details/4950918674?utm_medium=api&utm_source=55ce14f0

Title: Software Developer
Company: rocky recruiters
Location: Australia, New South Wales, Sydney Region, Sydney, The Rocks
Link: https://www.adzuna.com.au/details/4907434017?utm_medium=api&utm_source=55ce14f0

Title: So